In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import numpy as np
import matplotlib.pyplot as plt

# Establecemos un estilo para mejorar la estética.
plt.style.use('ggplot')
plt.rcParams.update({
    'font.size': 12,
    'axes.labelsize': 14,
    'axes.titlesize': 16,
    'legend.fontsize': 12,
    'figure.figsize': (8, 6),
    'lines.linewidth': 2,
    'grid.linestyle': '--',
    'grid.alpha': 0.6
})

# Creamos la carpeta "figuras" donde se guardarán las gráficas.
FIGURES_DIR = "figuras"
os.makedirs(FIGURES_DIR, exist_ok=True)

def read_xvg(filename):
    """
    Lee un archivo .xvg de Gromacs ignorando líneas que empiecen por '#' o '@'.
    Devuelve un np.array con los datos numéricos.
    """
    data = []
    with open(filename, 'r') as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith('#') or line.startswith('@'):
                continue
            parts = line.split()
            try:
                data.append([float(x) for x in parts])
            except ValueError:
                continue
    return np.array(data)

def read_ramach_dat(filename):
    """
    Lee un archivo .dat para diagramas de Ramachandran.
    Se espera que cada línea tenga: phi psi ResidueLabel.
    Devuelve un np.array de shape (N,2) con las dos primeras columnas (phi y psi).
    """
    data = []
    with open(filename, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split()
            # Se esperan al menos 2 valores numéricos en las dos primeras columnas.
            try:
                phi = float(parts[0])
                psi = float(parts[1])
                data.append([phi, psi])
            except ValueError:
                continue
    return np.array(data)

def plot_histogram_temp(temp_data, outname='temp_hist_500ps.png'):
    """
    Genera un histograma de la temperatura (suponemos que la segunda columna de temp.xvg es la temperatura).
    """
    temperature = temp_data[:, 1]
    plt.figure()
    plt.hist(temperature, bins=50, color='skyblue', edgecolor='black')
    plt.xlabel('Temperatura (K)')
    plt.ylabel('Frecuencia')
    plt.title('Histograma de Temperatura (500 ps)')
    plt.tight_layout()
    plt.savefig(os.path.join(FIGURES_DIR, outname), dpi=300)
    plt.close()

def plot_velocity_histograms(vel_data, outname='veloc_hist_500ps.png'):
    """
    Genera histogramas de las tres componentes de velocidad (vx, vy, vz)
    en un mismo lienzo. Se asume que vel_data tiene al menos 4 columnas:
    tiempo, vx, vy, vz.
    """
    if vel_data.shape[1] < 4:
        print("ERROR: El archivo de velocidades no tiene al menos 4 columnas.")
        return
    vx = vel_data[:, 1]
    vy = vel_data[:, 2]
    vz = vel_data[:, 3]

    fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    
    axs[0].hist(vx, bins=50, color='salmon', edgecolor='black')
    axs[0].set_xlabel('vx (nm/ps)')
    axs[0].set_ylabel('Frecuencia')
    axs[0].set_title('Histograma vx (500 ps)')
    
    axs[1].hist(vy, bins=50, color='lightgreen', edgecolor='black')
    axs[1].set_xlabel('vy (nm/ps)')
    axs[1].set_ylabel('Frecuencia')
    axs[1].set_title('Histograma vy (500 ps)')
    
    axs[2].hist(vz, bins=50, color='lightblue', edgecolor='black')
    axs[2].set_xlabel('vz (nm/ps)')
    axs[2].set_ylabel('Frecuencia')
    axs[2].set_title('Histograma vz (500 ps)')
    
    plt.tight_layout()
    plt.savefig(os.path.join(FIGURES_DIR, outname), dpi=300)
    plt.close()

def plot_rama_2d_hist(rama_data, residue, outname='rama_2dhist.png'):
    """
    Genera un diagrama de Ramachandran en forma de histograma 2D.
    rama_data debe ser un np.array de shape (N,2) con φ y ψ.
    """
    phi = rama_data[:, 0]
    psi = rama_data[:, 1]
    
    plt.figure()
    # Se definen 72 bins para cubrir el rango [-180, 180] en pasos de 5°.
    h = plt.hist2d(phi, psi, bins=[72,72], range=[[-180, 180], [-180, 180]], cmap='viridis')
    plt.colorbar(label='Frecuencia')
    plt.xlabel(r'$\phi$ (grados)')
    plt.ylabel(r'$\psi$ (grados)')
    plt.title(f'Diagrama Ramachandran 2D ({residue}, 500 ps)')
    plt.xlim([-180, 180])
    plt.ylim([-180, 180])
    plt.tight_layout()
    plt.savefig(os.path.join(FIGURES_DIR, outname), dpi=300)
    plt.close()

def main():
    # 1. Histograma de Temperatura
    temp_data = read_xvg('temp.xvg')
    plot_histogram_temp(temp_data, outname='temp_hist_500ps.png')
    
    # 2. Histogramas de Velocidades
    vel_data = read_xvg('veloc.xvg')
    plot_velocity_histograms(vel_data, outname='veloc_hist_500ps.png')
    
    # 3. Diagramas de Ramachandran 2D para cada residuo
    # Se asume que se tienen archivos: ala-2.dat, arg-3.dat y gly-4.dat
    ala2_data = read_ramach_dat('ala-2.dat')
    plot_rama_2d_hist(ala2_data, residue='ALA-2', outname='rama_ala2_2dhist.png')
    
    arg3_data = read_ramach_dat('arg-3.dat')
    plot_rama_2d_hist(arg3_data, residue='ARG-3', outname='rama_arg3_2dhist.png')
    
    gly4_data = read_ramach_dat('gly-4.dat')
    plot_rama_2d_hist(gly4_data, residue='GLY-4', outname='rama_gly4_2dhist.png')
    
    print("¡Gráficas generadas en la carpeta 'figuras' para la simulación de 500 ps!")

if __name__ == '__main__':
    main()


¡Gráficas generadas en la carpeta 'figuras' para la simulación de 500 ps!
